# OpenAI Basics

In [ ]:
!pip install -q requests == 2.32.3
!pip install -q openai == 1.41.1

In [ ]:
import requests
import json

from openai import OpenAI
from google.colab import userdata

## Task1: Generate OpenAI key

Generate key from: https://platform.openai.com/api-keys

In [ ]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
model = 'gpt-3.5-turbo'
messages = [
        {'role': 'system', 'content': 'You are a helpful assistant.'}, # system prompt
        {'role': 'user', 'content': 'Hello!'} # user prompt
    ]

## Task2: Call OpenAI using requests lib

In [ ]:
url = 'https://api.openai.com/v1/chat/completions'
headers = {
    'Authorization': f'Bearer {OPENAI_API_KEY}',
    'Content-Type': 'application/json'
}

body = {
    'model': model,
    'messages': messages
}

body_json = json.dumps(body)
response = requests.post(url = url, headers = headers, body = body_json)
print(f'resp status code: {response.status_code}, resp body: {response.json()}')

## Task3: Call OpenAI using OpenAI Python library

In [ ]:
client = OpenAI(api_key = OPENAI_API_KEY)
completion = client.chat.completions.create(
    model = model,
    messages = messages,
    temperature = 0 # default = 1
)

print(completion.choices[0].message)

## Task4: Building basic LLM App

### 4.1: Reviews Tagging Application

This application will automatically analyze and tag customer reviews based on their content, providing valuable insights for businesses.

In [ ]:
system_prompts = """
You will be presented with user reviews and your job is to provide a set of tags from the following list.
Provide your answer in json format with key "tags" and value is a set of tags. Choose ONLY from the list of tags provided here
(choose either the positive or the negative tag but NOT both):
  - Provides good value for the price OR Costs too much
  - Works better than expected OR Did not work as well as expected
  - Includes essential features OR Lacks essential features
  - Easy to use OR Difficult to use
  - High quality and durability OR Poor quality and durability
  - Easy and affordable to maintain or repair OR Difficult or costly to maintain or repair
  - Easy to transport OR Difficult to transport
  - Easy to store OR Difficult to store
  - Compatible with other devices or systems OR Not compatible with other devices or systems
  - Safe and user-friendly OR Unsafe or hazardous to use
  - Excellent customer support OR Poor customer support
  - Generous and comprehensive warranty OR Limited or insufficient warranty
"""

def analyze_review(text):
    response = client.chat.completions.create(
        model = model,
        messages=[
            {'role': 'system', 'content': [{'type': 'text', 'text': system_prompts}]},
            {'role': 'user', 'content': [{'type': 'text', 'text': text}]}
        ],
        temperature = 0,
        max_tokens = 256,
        top_p = 1,
        frequency_penalty = 0,
        presence_penalty = 0,
        response_format = {'type': 'text'}
    )

    return json.loads(response.choices[0].message.content)

In [ ]:
analyze_review('The product is great, very easy to use!')

### 4.2: ENG Language Level Augmenter Application

This tool will allow users to convert text between different levels of formality

In [ ]:
style_prompts = {
    'informal': 'Using informal words like talking to a friend.',
    'ielts': 'Using very fancy words.',
    'formal': 'Using formal words.',
    'academic': 'Using academic words suitable for academic publications.',
    'default': ''
}

def augment_sentence(text, style = "default"):
    content = f'You will be provided with statements, and your task is to convert them to standard English. {style_prompts[style]}'
    response = client.chat.completions.create(
      model = model,
      messages=[
          {'role': 'system', 'content': content},
          {'role': 'user', 'content': text}
      ],
      temperature=0.7,
      max_tokens=64,
      top_p=1
    )

    return response.choices[0].message.content

In [ ]:
test_text = """
I am writing this email to apply for a job at KBTG in software developer position.
I'm currently a student at 4 year univercity. I want this job very much.
"""
augment_sentence(test_text, style = 'formal')